# Billboard Top 100 Songs

In [12]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [11]:
response = requests.get('https://www.billboard.com/charts/hot-100')
doc = BeautifulSoup(response.content, 'html.parser')

rankList = []
songList = []
artistList = []

ranks = doc.find_all(class_='chart-list-item__rank')
for rank in ranks:
    rankList.append(rank.text.strip())

songs = doc.find_all(class_= 'chart-list-item__title-text')
for song in songs:
    songList.append(song.text.strip())
    
artists = doc.find_all(class_='chart-list-item__artist')
for artist in artists:
    artistList.append(artist.text.strip())

#OUTPUT CSV
total = dict(zip(['RANK', 'SONG', 'ARTIST'],[rankList, songList, artistList]))
table = pd.DataFrame(total)
table.to_csv("BillboardTop100Songs.csv", index=False)

#this website must uses (response.content, 'html.parser')
#check the doc before choosing html.parser or text

# Science Fiction Books by Female Authors

In [2]:
response = requests.get('https://www.goodreads.com/list/show/6934.Science_Fiction_Books_by_Female_Authors')
books = BeautifulSoup(response.text)
books = books.find(class_='tableList js-dataTooltip')

rankList = []
titleList = []
authorList = []
scoreList = []
voteList = []
ratingList = []

ranks = books.find_all(class_='number')
for rank in ranks:
    rankList.append(rank.text.strip())

titles = books.find_all(class_='bookTitle')
for title in titles:
    titleList.append(title.text.strip())
    
authors = books.find_all(attrs={"itemprop": "author"})
for author in authors:
    authorList.append(author.text.strip())
    
rates = books.find_all(class_='minirating')
for rate in rates:
    ratingList.append(rate.text.strip())
    
votes = books.find_all(class_='smallText uitext')
for vote in votes:
    text = vote.text.strip().replace('\n', '').replace('\n', '')
    #Split the text, find the socre
    position = text.find(',              and')
    scoreList.append(text[0:position])
    #Split the text, find the vulue of vote
    voteList.append(text[position+18:-1])

#OUTPUT CSV
total = dict(zip(['RANK', 'TITLE', 'AUTHOR', 'SCORE', 'VOTES', 'RATING'],[rankList, titleList, authorList, scoreList, voteList, ratingList]))
table = pd.DataFrame(total)
table.to_csv("ScienceFictionBooks.csv", index=False)

# Science Fiction Books by Female Authors (Clean Version)

In [3]:
response = requests.get('https://www.goodreads.com/list/show/6934.Science_Fiction_Books_by_Female_Authors')
books = BeautifulSoup(response.text)

rankList = []
titleList = []
seriesList = []
seriesNumList = []
authorList = []
scoreList = []
voteList = []
ratingList = []
ratingNumList = []

#RANKS' PART
ranks = books.find_all(class_='number')
for rank in ranks:
    rankList.append(rank.text.strip())

#TITLES' PART
titles = books.find_all(class_='bookTitle')
for title in titles:
    title = title.text.strip()
    #If the book doesn't belong to any series
    if (title.find('(') == -1):
        titleList.append(title)
        seriesList.append('None')
        seriesNumList.append('None')
    else:
        #Find the position of parentheses
        l_position = title.find('(')
        if (title.find(', #') != -1): 
            p_position = title.find(', #')
            detract = 3
        else:
            p_position = title.find('#')
            detract = 1
        r_position = title.find(')')
        #Write them into lists
        titleList.append(title[0:l_position].strip())
        seriesList.append(title[l_position+1:p_position].strip())
        seriesNumList.append(title[p_position+detract:r_position].strip())

#AUTHOR'S PART
authors = books.find_all(attrs={"itemprop": "author"})
for author in authors:
    authorList.append(author.text.strip())

#RATES' PART
rates = books.find_all(class_='minirating')
for rate in rates:
    rate = rate.text.strip()
    
    #If the text includ "really liked it", remove it
    if (rate.find('really liked it') != -1):
        rate = rate.replace('really liked it','').strip()
    
    #Get the positions
    l_position = rate.find(' avg')
    c_position = rate.find('—')
    r_position = rate.rfind(' ratings')
    
    #Write them into lists
    ratingList.append(rate[0:l_position].replace(',',''))
    ratingNumList.append(rate[c_position+2:r_position].replace(',',''))

#VOTES' PART
votes = books.find_all(class_='smallText uitext')
for vote in votes:
    text = vote.text.replace('\n', '').replace('\n', '')
    #Split the text, find the socre
    position = text.find(',              and')
    scoreList.append(text[7:position])
    #Split the text, find the vote vulue
    preVote = text[position+18:-1]
    votePosition = preVote.find(' people voted')
    voteList.append(preVote[0:votePosition])

#OUTPUT CSV
total = dict(zip(['RANK', 'TITLE', 'SERIES', 'NUMBER IN SERIES', 'AUTHOR', 'SCORE', 'AVG RATINGS','NUM OF RATINGS', 'VOTES'],[rankList, titleList, seriesList, seriesNumList, authorList, ratingList, ratingNumList, voteList]))
table = pd.DataFrame(total)
table.to_csv("ScienceFictionBooks_Clean.csv", index=False)

#why the first one Billboard sometimes doesnot produce lists?

# Epicurious Part1

In [4]:
#Set request times
times = 3

#Set a search keyword
keyword = 'cucumbers'

#Initialize lists
categoryList = []
titleList = []
summaryList = []
ratingList = []
againList = []
urlList = []

#Initialize search results
finalResult = ""

#Start requesting result
for i in range(1,times):
    
    #Get the result list
    page = requests.get("https://www.epicurious.com/search/" + keyword + "?page=" + str(i))
    result = BeautifulSoup(page.text)
    result = result.find(attrs={"data-group-number": i})
    
    #Combine all results together
    finalResult += str(result)

#Search the final result
epicurious = BeautifulSoup(finalResult)

#Get the categories list
categories = epicurious.find_all(class_="tag")
for category in categories:
    category = category.text.strip()
    categoryList.append(category)
    
#Get the title list
titles = epicurious.find_all(attrs={"itemprop": "name"})
for title in titles:
    title = title.text.strip()
    titleList.append(title)

#Get the summary list
summaries = epicurious.find_all(class_="summary")
for summary in summaries:
    summary = summary.find(class_="dek").text
    summaryList.append(summary)
    
#Get the rating list
ratings = epicurious.find_all(class_="summary")
for rating in ratings:
    try:
        rating = rating.find(attrs={"itemprop": "ratingValue"}).text
    except:
        rating = "N/A"
    ratingList.append(rating)

#Get the "Make it again" list
agains = epicurious.find_all(class_="summary")
for again in agains:
    try: 
        again = again.find(class_="make-again-percentage").text
    except:
        again = "N/A"
    againList.append(again)

#Get the URL list
urls = epicurious.find_all(class_="hed")
for url in urls:
    url = url.find("a").get('href')
    urlList.append("https://www.epicurious.com"+url)
      
#Output CSV
total = dict(
    zip(["Category", 
         "Title", 
         "Summary", 
         "Rating", 
         "Make it again", 
         "URL"],
        [categoryList,
         titleList, 
         summaryList, 
         ratingList, 
         againList, 
         urlList]))
table = pd.DataFrame(total)
table.to_csv("Epicurious.csv", index=False)

# Epicurious Part2

In [9]:
#Set the category to filter, recipe or article
keyword = "recipe"

#Set the file location
file = "Epicurious.csv"

#Read CSV
df = pd.read_csv(file)

#Filter 
results = df[df["Category"] == keyword].values

#Initialize lists
categoryList = []
titleList = []
summaryList = []
ratingList = []
againList = []
urlList = []
stepList = []
ingList = []
tagList = []

#Featching ingredients, directions and tags
for i in results:
    page = requests.get(i[5])
    result = BeautifulSoup(page.text)
    result = result.find(class_="recipe-and-additional-content")
    
    #Rereade data
    categoryList.append(i[0])
    titleList.append(i[1])
    summaryList.append(i[2])
    ratingList.append(i[3])
    againList.append(i[4])
    urlList.append(i[5])
    
    #Initialize ingredients(temporary)
    t_ingredients = ""
    #Scrap ingredients
    ingredients = result.find(class_="ingredients")
    t_ingredients += '\n'.join([ingredient.text for ingredient in ingredients])
    #Write them into ingList
    ingList.append(t_ingredients)
    
    #Initialize steps(temporary)
    #I didn't find direction, so guess it should be preparation? 
    t_steps = ""
    #Scrap steps
    steps = result.find(class_="preparation-steps")
    t_steps += "\n".join([step.text.strip() for step in steps])
    #Write them into stepList
    stepList.append(t_steps)
    
    #Initialize tags(temporary)
    t_tags = ""
    #Scrap tags
    tags = result.find(class_="tags")
    t_tags += ",".join([tag.text.strip() for tag in tags])
    #Write them into tagList
    tagList.append(t_tags)

#Output CSV
total = dict(
    zip(["Category", 
         "Title", 
         "Summary",
         "Rating", 
         "Make it again", 
         "Ingredient",
         "Step",
         "Tag",
         "URL"],
        [categoryList,
         titleList, 
         summaryList, 
         ratingList, 
         againList, 
         ingList,
         stepList,
         tagList,
         urlList]))
table = pd.DataFrame(total)
table.to_csv("Epicurious_Recips_Only.csv", index=False)

# Metrolyrics Part1

In [10]:
#Set a singer page
singer = "Mariah Carey"

#Initialize 
songList = []
yearList = []
popList = []
urlList = []
singerName = singer.replace(" ","-")

#Count pages
def countNum (singerName):
    lastNum = 0
    stopSearch = 1
    while stopSearch == 1:
        if lastNum == 0:
            page = requests.get("http://www.metrolyrics.com/" + singerName +"-lyrics.html")
            result = BeautifulSoup(page.text)
        else:
            page = requests.get("http://www.metrolyrics.com/" + singerName +"-alpage-" + str(lastNum) + ".html")
            result = BeautifulSoup(page.text)
            
        #If it's the last page
        if (result.find("a", class_="button next disabled") == None):
            
            #If it's not the last page but including "...", get the last num and keep searching
            if (result.find(text=" ... ") != None):
                url = result.find("span", "pages").find_all("a")
                url = url[len(url)-1].get('href')
                #find the position
                l_position = url.find("alpage-")
                r_position = url.find(".html")
                lastNum = url[l_position+7:r_position]
                stopSearch = 1
                
            #If it's not the last page and not including "...", get the last num and stop searching 
            else:
                lastNum = result.find("span", "pages").find_all("a")
                lastNum = lastNum[len(lastNum)-1].text.strip()
                stopSearch = 0
                
        #If the NEXT button is disabled, get the last num and stop searching
        elif (result.find("a", class_="button next disabled") != None) :
            lastNum = result.find("span", "pages").find_all("a")
            lastNum = lastNum[len(lastNum)-1].text.strip()
            stopSearch = 0
        else:
            print("Something went wrong!")
            break
    return int(lastNum)

for i in range(1,countNum(singerName)+1):
    #Featch data
    page = requests.get("http://www.metrolyrics.com/" + singerName +"-alpage-" + str(i) + ".html")
    result = BeautifulSoup(page.text)
    result = result.find(class_="songs-table compact")

    #Scrap songs' title and URL
    songs = result.find_all(class_="title")
    for song in songs:
        url = song.get('href')
        urlList.append(url)
        song = song.text.strip()
        song = song.replace(" Lyrics","")
        songList.append(song)

    #Scrap years
    years = result.find("tbody").find_all("tr")
    for year in years:
        year = year.find("td").next_sibling.next_sibling.next_sibling.next_sibling.text
        yearList.append(year)

    #Scrap popularity
    pops = result.find("tbody").find_all("tr")
    for pop in pops:
        pop = pop.find(class_="bar").find("span")
        pop = pop["style"].replace("width:","").replace("%;","").strip()
        pop = "{:.0f}".format(float(pop))
        popList.append(pop)

#Output CSV
total = dict(
    zip(["Song", 
         "Year", 
         "Popularity",
         "URL"
         ],
        [songList,
         yearList, 
         popList, 
         urlList]))
table = pd.DataFrame(total)
table.to_csv("Metrolyrics-" + singerName + ".csv", index=False)

# Metrolyrics Part2

In [7]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

#Set the file location
file = "Metrolyrics-Mariah-Carey.csv"

#Read CSV
df = pd.read_csv(file) 

#Only fetch top 5 songs for saving memory
results = df.head(5).values

#For using the results list, those numbers:
    #0 SongName
    #1 Year
    #2 Popularity
    #3 URL

#Initialize 
songList = []
yearList = []
popList = []
urlList = []    
lyricList = []

#Featching lyrics
for i in results: 
    page = requests.get(i[3])
    
    #Rereade data
    songList.append(i[0])
    yearList.append(i[1])
    popList.append(i[2])
    urlList.append(i[3])
    
    #Read the URL
    result = BeautifulSoup(page.text)
    
    #Read lyric
    t_lyric = ""
    lines = result.find_all(class_="verse")
    t_lyric = "\n".join([line.text for line in lines])
    lyricList.append(t_lyric)

#Output CSV
total = dict(
    zip(["Song", 
         "Year", 
         "Popularity",
         "Lyric",
         "URL"],
        [songList,
         yearList, 
         popList, 
         lyricList,
         urlList]))
table = pd.DataFrame(total)
table.to_csv(file.replace(".csv", "-lyrics.csv"), index=False)